In [1]:
import keras 
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras import optimizers
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                shear_range = 0.2,
                                   zoom_range = 0.2,
                                   validation_split=0.2,
                                   horizontal_flip = True)


training_set = train_datagen.flow_from_directory(r'C:\projects_star\fakeindiancurrency\Indain Currency dataset\train',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 subset="training",
                                                 class_mode = 'categorical')

validation_set = train_datagen.flow_from_directory(r'C:\projects_star\fakeindiancurrency\Indain Currency dataset\test',
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                subset="validation",
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'C:\projects_star\fakeindiancurrency\Indain Currency dataset\test',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

#color_mode = "grayscale"


STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
STEP_SIZE_VALID=validation_set.n//validation_set.batch_size
STEP_SIZE_TEST=test_set.n//test_set.batch_size

Found 928 images belonging to 2 classes.
Found 107 images belonging to 2 classes.
Found 535 images belonging to 2 classes.


In [3]:
IMAGE_SIZE = [224, 224]
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#here [3] denotes for RGB images(3 channels)

#don't train existing weights
for layer in vgg.layers:
 layer.trainable = False
 
x = Flatten()(vgg.output)
prediction = Dense(2, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(loss='binary_crossentropy',
                    optimizer=optimizers.Adam(),
                    metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
checkpoint = ModelCheckpoint(filepath='mymodel1.h5', 
                               verbose=1, save_best_only=True)
callbacks = [checkpoint, lr_reducer]
start = datetime.now()
# history = model.fit_generator(training_set, 
#                     steps_per_epoch=STEP_SIZE_TRAIN, 
#                     epochs = 18, verbose=5, 
#                     validation_data = validation_set, 
#                     validation_steps = STEP_SIZE_VALID)
# duration = datetime.now() - start
# print("Training completed in time: ", duration)

# Assuming you have a `training_set` generator
# and `batch_size` is the batch size used in the generator

batch_size=8
total_samples = len(training_set) * batch_size
steps_per_epoch = total_samples // batch_size

# Make sure steps_per_epoch is at least 1
steps_per_epoch = max(steps_per_epoch, 1)

history = model.fit_generator(
    training_set,
    steps_per_epoch=steps_per_epoch,
    epochs=18,
    verbose=5,
    validation_data=validation_set,
    validation_steps=len(validation_set),  # or calculate based on validation data size
    callbacks=[checkpoint, lr_reducer]
)
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/18


C:\Users\aksha\AppData\Local\Temp\ipykernel_7492\3655621403.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(



Epoch 1: val_loss improved from inf to 0.46127, saving model to mymodel1.h5
Epoch 2/18


c:\Users\aksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 0.46127 to 0.34786, saving model to mymodel1.h5
Epoch 3/18

Epoch 3: val_loss improved from 0.34786 to 0.23617, saving model to mymodel1.h5
Epoch 4/18

Epoch 4: val_loss did not improve from 0.23617
Epoch 5/18

Epoch 5: val_loss improved from 0.23617 to 0.18353, saving model to mymodel1.h5
Epoch 6/18

Epoch 6: val_loss improved from 0.18353 to 0.13996, saving model to mymodel1.h5
Epoch 7/18

Epoch 7: val_loss did not improve from 0.13996
Epoch 8/18

Epoch 8: val_loss improved from 0.13996 to 0.12054, saving model to mymodel1.h5
Epoch 9/18

Epoch 9: val_loss did not improve from 0.12054
Epoch 10/18

Epoch 10: val_loss did not improve from 0.12054
Epoch 11/18

Epoch 11: val_loss did not improve from 0.12054
Epoch 12/18

Epoch 12: val_loss improved from 0.12054 to 0.06850, saving model to mymodel1.h5
Epoch 13/18

Epoch 13: val_loss improved from 0.06850 to 0.05612, saving model to mymodel1.h5
Epoch 14/18

Epoch 14: val_loss did not improve from 0.05612
Epo

In [5]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

67/67 [==============================] - 35s 523ms/step - loss: 0.0450 - accuracy: 0.9888
Test Loss: 0.04496966674923897
Test accuracy: 0.9887850284576416


In [10]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained models
model= load_model('mymodel1.h5')  # Trained model for spiral dataset
# model_wave = load_model('mymodel_wave.h5')  # Trained model for wave dataset

# Path to the new image
new_image_path = r"C:\Users\aksha\Downloads\test (3)_aug_1.jpg" # Replace with the path to your new image

# Load and preprocess the new image
new_img = image.load_img(new_image_path, target_size=(224, 224))
new_img_array = image.img_to_array(new_img)
new_img_array = np.expand_dims(new_img_array, axis=0)
new_img_array /= 255.0  # Normalize the image

# Make predictions using both models
prediction= model.predict(new_img_array)

# Convert the combined predicted probabilities to class labels
predicted_class = np.argmax(prediction)

# Define the class labels
class_labels = ['Fake', 'Real']

# Get the label of the image
predicted_label = class_labels[predicted_class]

# Display the predicted label
print("Predicted Label:", predicted_label)


1/1 [==============================] - 0s 258ms/step
Predicted Label: Real
